# Scraping

This notebook follows an example of scraping Nadlan Gov. It yeilds a query csv in 'data/gov' folder which will be avilable of off line analysis. 

Follow the comments to modify and try different queries.

**Note:** This proceedure takes time, queries with a mutiple results could take a long time. Try not to query about more than one neigberhood.

In [346]:
import time

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service

# import org.openqa.selenium.JavascriptExecutor as JavascriptExecutor
import pprint
import pandas as pd

In [347]:
# insert your q
q = '4000'


In [348]:
# update to your driver path
DRIVER_PATH = 'C:/Users/IMOE001/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe'

url = f'https://www.nadlan.gov.il/?view=settlement&id={q}&page=deals'

service = Service(DRIVER_PATH)
browser = webdriver.Chrome(service=service)
browser.get(url)

# now wait for load
browser.implicitly_wait(5)

browser.set_window_size(1500, 1000)

In [349]:
class infinite_scroll(object):
   
    def __init__(self, last):
    
        self.last = last

    def __call__(self, driver):
        new = driver.execute_script('return document.body.scrollHeight')  
        if new > self.last:
            return new
        else:
            return False
    

In [350]:
last_height = browser.execute_script('return document.body.scrollHeight')
print(last_height)
flag=1

while flag==1:
    browser.execute_script('window.scrollTo(0,document.body.scrollHeight)')

    try:
        w = wait(browser, 10)

        new_height = w.until(infinite_scroll( last_height))
        last_height = new_height

    except:
        print("End of page reached")
        flag = 0

2647
End of page reached


In [351]:
relevant_col_names = ['קומות במבנה','מחיר למ"ר','שנת בנייה','תאריך עסקה', 'כתובת', 'גוש/חלקה/תת-חלקה', 'סוג נכס', 'חדרים', 'קומה', 'מ"ר','מחיר']
relevant_col_dict = {c:[] for c in relevant_col_names}

In [352]:
def safe_get(features, idx):
    return features[idx].text if len(features) > idx else ""

In [ ]:
has_next = True

while has_next:
    table = browser.find_elements(By.CLASS_NAME, "mainTable")[0]
    sell_row_data = table.find_elements(By.CLASS_NAME, "mainTable__row")

    for i in range(1, len(sell_row_data)):
        ActionChains(browser).move_to_element(sell_row_data[i]).perform()

        features = sell_row_data[i].find_elements(By.CLASS_NAME,"mainTable__cell")
        relevant_col_dict['כתובת'].append(safe_get(features, 1))
        relevant_col_dict['מ"ר'].append(safe_get(features, 2))
        relevant_col_dict['תאריך עסקה'].append(safe_get(features, 3))
        relevant_col_dict['מחיר'].append(safe_get(features, 4))
        relevant_col_dict['גוש/חלקה/תת-חלקה'].append(safe_get(features, 5))
        relevant_col_dict['סוג נכס'].append(safe_get(features, 6))
        relevant_col_dict['חדרים'].append(safe_get(features, 7))
        relevant_col_dict['קומה'].append(safe_get(features, 8))

        # Click the collapse arrow using JS to avoid interception
        arrow = sell_row_data[i].find_elements(By.CLASS_NAME, "collapseArrow")[0]
        browser.execute_script("arguments[0].click();", arrow)
        time.sleep(0.5)  # Give time for the DOM to update

        sell_row_data_collapse = table.find_elements(By.CLASS_NAME, "innerTablesContainer")[0]
        features = sell_row_data_collapse.find_elements(By.CLASS_NAME,"innerTable__cell")
        # get previous deals
        # print(safe_get(features, 8), safe_get(features, 9...)
        relevant_col_dict['שנת בנייה'].append(safe_get(features, 3))
        relevant_col_dict['מחיר למ"ר'].append(safe_get(features, 4))
        relevant_col_dict['קומות במבנה'].append(safe_get(features, 5))

        browser.execute_script("arguments[0].click();", arrow)
        time.sleep(0.5)  # Give time for the DOM to update

    next_button = browser.find_elements(By.ID, "next")
    if len(next_button) == 0:
        has_next = False
        break
    ActionChains(browser).move_to_element(next_button[0]).perform()
    browser.execute_script("arguments[0].click();", next_button[0])
    time.sleep(1)  # Give time for the DOM to update

print(relevant_col_dict)

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=137.0.7151.69); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff6dda9fea5+79173]
	GetHandleVerifier [0x0x7ff6dda9ff00+79264]
	(No symbol) [0x0x7ff6dd859e5a]
	(No symbol) [0x0x7ff6dd870820]
	(No symbol) [0x0x7ff6dd86ed33]
	(No symbol) [0x0x7ff6dd862551]
	(No symbol) [0x0x7ff6dd86041f]
	(No symbol) [0x0x7ff6dd86489c]
	(No symbol) [0x0x7ff6dd86496f]
	(No symbol) [0x0x7ff6dd8b7954]
	(No symbol) [0x0x7ff6dd8b6b8b]
	(No symbol) [0x0x7ff6dd90c75d]
	(No symbol) [0x0x7ff6dd8d896a]
	(No symbol) [0x0x7ff6dd90100d]
	(No symbol) [0x0x7ff6dd8d8743]
	(No symbol) [0x0x7ff6dd8a14c1]
	(No symbol) [0x0x7ff6dd8a2253]
	GetHandleVerifier [0x0x7ff6ddd6a2dd+3004797]
	GetHandleVerifier [0x0x7ff6ddd6472d+2981325]
	GetHandleVerifier [0x0x7ff6ddd83380+3107360]

In [ ]:
nadlan_df = pd.DataFrame.from_dict(relevant_col_dict)

for k,v in relevant_col_dict.items():
    print(f'{k}: {len(v)}')

קומות במבנה: 10
מחיר למ"ר: 10
שנת בנייה: 10
תאריך עסקה: 10
כתובת: 10
גוש/חלקה/תת-חלקה: 10
סוג נכס: 10
חדרים: 10
קומה: 10
מ"ר: 10
מחיר: 10


In [ ]:
# name your csv
csv_name = 'Haifa'

nadlan_df.to_csv(f'data/gov/{csv_name}.csv', index=False)